In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%capture

!pip install /kaggle/input/rsnamodules/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl 

try:
    import pylibjpeg
except:
   !pip install /kaggle/input/rsna-2022-whl/{pylibjpeg-1.4.0-py3-none-any.whl,python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl}

In [ ]:
!pip install -qU python-gdcm pydicom pylibjpeg

In [ ]:
pip install dicomsdl

In [ ]:
import os
import cv2
import glob
import gdcm
import pydicom
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
import dicomsdl as dicoml

In [ ]:
train = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')
train.head()

In [ ]:
train_images=glob.glob('/kaggle/input/rsna-breast-cancer-detection/train_images/*/*.dcm')
len(train_images)

In [ ]:
image_dir_dicomsdl = '/kaggle/working/png_file_dic/'
os.makedirs(image_dir_dicomsdl, exist_ok=True)

In [ ]:
def process(f, size=512, save_folder=None, dicom_process = True, extension="png"):
    
    patient = f.split('/')[-2]
    image_name = f.split('/')[-1][:-4]
    if dicom_process:
        dicom = pydicom.dcmread(f)
        img = dicom.pixel_array

        img = (img - img.min()) / (img.max() - img.min())

        if dicom.PhotometricInterpretation == "MONOCHROME1":  
            img = 1 - img
            
        image = (img * 255).astype(np.uint8)
    else:
        
        dicom = dicoml.open(f)
        img = dicom.pixelData()

        img = (img - img.min()) / (img.max() - img.min())

        if dicom.getPixelDataInfo()['PhotometricInterpretation'] == "MONOCHROME1":
            img = 1 - img

        image = (img * 255).astype(np.uint8)
    
    img = cv2.resize(image, (size, size))

    file_name = f'{save_folder}' + f"{patient}_{image_name}.{extension}"

    cv2.imwrite(file_name, img)

In [ ]:
Parallel(n_jobs=4)(
    delayed(process)(f, size = 512, save_folder = image_dir_dicomsdl, dicom_process = False)
    for f in tqdm(train_images)
)